In [1]:
import pandas as pd
import sys
sys.path.append("../helper_functions")
import clickhouse_utils as ch
import opstack_metadata_utils as ops
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()

True

In [2]:
# Get Chain List
chain_configs = ops.get_superchain_metadata_by_data_source('oplabs') # OPLabs db


In [3]:
# chain_configs

In [4]:
# Function to create ClickHouse view
def get_chain_names_from_df(df):
    return df['blockchain'].dropna().unique().tolist()

# Function to create ClickHouse view
def create_clickhouse_view(view_slug, dataset_type, chain_names, client = None):
    if client is None:
        client = ch.connect_to_clickhouse_db()

    query = f"CREATE OR REPLACE VIEW {view_slug}_{dataset_type} AS\n"
    union_queries = []
    
    for chain in chain_names:
        table_name = f"{chain}_{dataset_type}"
        if dataset_type == 'transactions':
            union_queries.append(f"""
                                SELECT 
                                id, hash, nonce, block_hash, block_number, transaction_index, from_address, to_address
                                , value, gas, gas_price, input, max_fee_per_gas, max_priority_fee_per_gas, transaction_type
                                , block_timestamp, receipt_cumulative_gas_used, receipt_gas_used, receipt_contract_address
                                , receipt_status, receipt_l1_fee, receipt_l1_gas_used, receipt_l1_gas_price, receipt_l1_fee_scalar
                                , receipt_l1_blob_base_fee, receipt_l1_blob_base_fee_scalar, blob_versioned_hashes, max_fee_per_blob_gas
                                , receipt_l1_block_number, receipt_l1_base_fee_scalar, chain, network, chain_id, insert_time
                                FROM {table_name} WHERE is_deleted = 0
                                """)
        else: 
            union_queries.append(f"""
                                SELECT 
                                *
                                FROM {table_name} WHERE is_deleted = 0
                                """)
    
    query += " UNION ALL\n".join(union_queries)

    # print(query)
    
    client.command(query)

    print(f"View '{view_slug}_{dataset_type}' created successfully.")

In [5]:
view_slug = 'superchain'
dataset_types = ['transactions', 'traces', 'blocks', 'logs']
chain_names = get_chain_names_from_df(chain_configs)
print(chain_names)
for dataset_type in dataset_types:
        create_clickhouse_view(view_slug, dataset_type, chain_names)

['op', 'zora', 'base', 'mode', 'fraxtal', 'metal', 'mint', 'cyber']
View 'superchain_transactions' created successfully.


DatabaseError: :HTTPDriver for https://pdmv9lhojy.us-west-2.aws.clickhouse.cloud:8443 returned response code 404)
 Code: 47. DB::Exception: Missing columns: 'max_fee_per_blob_gas' 'blob_versioned_hashes' 'receipt_l1_blob_base_fee' 'transaction_type' 'receipt_l1_fee_scalar' 'receipt_l1_fee' 'receipt_gas_used' 'receipt_status' 'receipt_cumulative_gas_used' 'receipt_l1_gas_price' 'receipt_l1_blob_base_fee_scalar' 'receipt_l1_base_fee_scalar' 'receipt_contract_address' 'receipt_l1_gas_used' 'receipt_l1_block_number' 'gas_price' 'hash' 'nonce' 'max_priority_fee_per_gas' 'max_fee_per_gas' while processing query: 'SELECT id, hash, nonce, block_hash, block_number, transaction_index, from_address, to_address, value, gas, gas_price, input, max_fee_per_gas, max_priority_fee_per_gas, transaction_type, block_timestamp, receipt_cumulative_gas_used, receipt_gas_used, receipt_contract_address, receipt_status, receipt_l1_fee, receipt_l1_gas_used, receipt_l1_gas_price, receipt_l1_fee_scalar, receipt_l1_